In [1]:
# import dependencies
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, balanced_accuracy_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

In [2]:
# read in our cleaned, merged data
ML_df = pd.read_csv('../resources/superstore.csv', index_col=[0])
ML_df

,Order Date,Ship Date,Ship Mode,Segment,City,State,Country,Region,Market,Category,...,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Returned,Profit Margin Percentage,Returned $ Amount,Days to Ship
Order ID,,,,,,,,,,,,,,,,,,,,,
CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,Consumer,Oklahoma City,Oklahoma,United States,Central US,USCA,Technology,...,221.98,2,0.0,62.15,40.770,High,0.0,27.998018,0.0,2
IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,Corporate,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,Furniture,...,3709.40,9,0.1,-288.77,923.630,Critical,0.0,-7.784817,0.0,2
IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,Consumer,Brisbane,Queensland,Australia,Oceania,Asia Pacific,Technology,...,5175.17,9,0.1,919.97,915.490,Medium,0.0,17.776614,0.0,1
ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,Home Office,Berlin,Berlin,Germany,Western Europe,Europe,Technology,...,2892.51,5,0.1,-96.54,910.160,Medium,0.0,-3.337586,0.0,2
SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,Consumer,Dakar,Dakar,Senegal,Western Africa,Africa,Technology,...,2832.96,8,0.0,311.52,903.040,Critical,0.0,10.996272,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IN-2015-KE1642066-42174,2015-06-19,2015-06-19,Same Day,Corporate,Kure,Hiroshima,Japan,Eastern Asia,Asia Pacific,Office Supplies,...,65.10,5,0.0,4.50,1.010,Medium,0.0,6.912442,0.0,0
US-2014-ZD21925140-41765,2014-05-06,2014-05-10,Standard Class,Consumer,Chattanooga,Tennessee,United States,Southern US,USCA,Furniture,...,16.72,5,0.2,3.34,1.930,High,0.0,19.976077,0.0,4
CA-2012-ZD21925140-41147,2012-08-26,2012-08-31,Second Class,Consumer,San Francisco,California,United States,Western US,USCA,Office Supplies,...,8.56,2,0.0,2.48,1.580,High,0.0,28.971963,0.0,5


# Preprocessing Data

In [3]:
# Order & Ship Date to datetime
ML_df['Order Date'] = pd.to_datetime(ML_df['Order Date'])
ML_df['Ship Date'] = pd.to_datetime(ML_df['Ship Date'])

# reset index & drop Order ID
ML_df = ML_df.reset_index()
ML_df = ML_df.drop(columns='Order ID')
ML_df

,Order Date,Ship Date,Ship Mode,Segment,City,State,Country,Region,Market,Category,...,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Returned,Profit Margin Percentage,Returned $ Amount,Days to Ship
0,2014-11-11,2014-11-13,First Class,Consumer,Oklahoma City,Oklahoma,United States,Central US,USCA,Technology,...,221.98,2,0.0,62.15,40.770,High,0.0,27.998018,0.0,2
1,2014-02-05,2014-02-07,Second Class,Corporate,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,Furniture,...,3709.40,9,0.1,-288.77,923.630,Critical,0.0,-7.784817,0.0,2
2,2014-10-17,2014-10-18,First Class,Consumer,Brisbane,Queensland,Australia,Oceania,Asia Pacific,Technology,...,5175.17,9,0.1,919.97,915.490,Medium,0.0,17.776614,0.0,1
3,2014-01-28,2014-01-30,First Class,Home Office,Berlin,Berlin,Germany,Western Europe,Europe,Technology,...,2892.51,5,0.1,-96.54,910.160,Medium,0.0,-3.337586,0.0,2
4,2014-11-05,2014-11-06,Same Day,Consumer,Dakar,Dakar,Senegal,Western Africa,Africa,Technology,...,2832.96,8,0.0,311.52,903.040,Critical,0.0,10.996272,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,2015-06-19,2015-06-19,Same Day,Corporate,Kure,Hiroshima,Japan,Eastern Asia,Asia Pacific,Office Supplies,...,65.10,5,0.0,4.50,1.010,Medium,0.0,6.912442,0.0,0
51286,2014-05-06,2014-05-10,Standard Class,Consumer,Chattanooga,Tennessee,United States,Southern US,USCA,Furniture,...,16.72,5,0.2,3.34,1.930,High,0.0,19.976077,0.0,4
51287,2012-08-26,2012-08-31,Second Class,Consumer,San Francisco,California,United States,Western US,USCA,Office Supplies,...,8.56,2,0.0,2.48,1.580,High,0.0,28.971963,0.0,5
51288,2013-02-17,2013-02-21,Standard Class,Home Office,Valinhos,S�o Paulo,Brazil,South America,LATAM,Office Supplies,...,13.44,2,0.0,2.40,1.003,Medium,0.0,17.857143,0.0,4


### Day of Week

In [4]:
# get day of week for order date
# runs 0 - 6; 0=Monday, 6=Sunday
ML_df['Order Day'] = ML_df['Order Date'].dt.dayofweek
ML_df['Order Day'].head()

0    1
1    2
2    4
3    1
4    2
Name: Order Day, dtype: int64

In [5]:
# move Order Day to separate df, will encode
day_dummies = pd.get_dummies(ML_df['Order Day']).reset_index()
day_dummies

,index,0,1,2,3,4,5,6
0,0,0,1,0,0,0,0,0
1,1,0,0,1,0,0,0,0
2,2,0,0,0,0,1,0,0
3,3,0,1,0,0,0,0,0
4,4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
51285,51285,0,0,0,0,1,0,0
51286,51286,0,1,0,0,0,0,0
51287,51287,0,0,0,0,0,0,1
51288,51288,0,0,0,0,0,0,1


In [6]:
# rename 0-6 index to days of week
day_dummies = day_dummies.drop(columns='index')
day_dummies.columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_dummies

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
51285,0,0,0,0,1,0,0
51286,0,1,0,0,0,0,0
51287,0,0,0,0,0,0,1
51288,0,0,0,0,0,0,1


In [7]:
# concat the encoded days of week to end of the ML_df
ML_df = ML_df.reset_index()
ML_df = pd.concat([ML_df, day_dummies], axis=1, join='inner')
ML_df

,index,Order Date,Ship Date,Ship Mode,Segment,City,State,Country,Region,Market,...,Returned $ Amount,Days to Ship,Order Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,2014-11-11,2014-11-13,First Class,Consumer,Oklahoma City,Oklahoma,United States,Central US,USCA,...,0.0,2,1,0,1,0,0,0,0,0
1,1,2014-02-05,2014-02-07,Second Class,Corporate,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,...,0.0,2,2,0,0,1,0,0,0,0
2,2,2014-10-17,2014-10-18,First Class,Consumer,Brisbane,Queensland,Australia,Oceania,Asia Pacific,...,0.0,1,4,0,0,0,0,1,0,0
3,3,2014-01-28,2014-01-30,First Class,Home Office,Berlin,Berlin,Germany,Western Europe,Europe,...,0.0,2,1,0,1,0,0,0,0,0
4,4,2014-11-05,2014-11-06,Same Day,Consumer,Dakar,Dakar,Senegal,Western Africa,Africa,...,0.0,1,2,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,51285,2015-06-19,2015-06-19,Same Day,Corporate,Kure,Hiroshima,Japan,Eastern Asia,Asia Pacific,...,0.0,0,4,0,0,0,0,1,0,0
51286,51286,2014-05-06,2014-05-10,Standard Class,Consumer,Chattanooga,Tennessee,United States,Southern US,USCA,...,0.0,4,1,0,1,0,0,0,0,0
51287,51287,2012-08-26,2012-08-31,Second Class,Consumer,San Francisco,California,United States,Western US,USCA,...,0.0,5,6,0,0,0,0,0,0,1
51288,51288,2013-02-17,2013-02-21,Standard Class,Home Office,Valinhos,S�o Paulo,Brazil,South America,LATAM,...,0.0,4,6,0,0,0,0,0,0,1


In [8]:
ML_df.columns

Index(['index', 'Order Date', 'Ship Date', 'Ship Mode', 'Segment', 'City',
       'State', 'Country', 'Region', 'Market', 'Category', 'Sub-Category',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost',
       'Order Priority', 'Returned', 'Profit Margin Percentage',
       'Returned $ Amount', 'Days to Ship', 'Order Day', 'Monday', 'Tuesday',
       'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype='object')

### Cleaning Up Columns

In [9]:
# pull year & month from the Order Date
ML_df['Order Year'] = ML_df['Order Date'].dt.year
ML_df['Order Month'] = ML_df['Order Date'].dt.month

# drop columns
# Order Day - encoded to separate columns
# Order Date - pulled to year & month
# Ship Date - redundant; order info & days to ship
# City & State & Country - too many unqiue values
# Market - redunant with Region (which is more specific)
ML_df = ML_df.drop(columns=['Order Day', 'Order Date', 'Ship Date', 'City', 'State', 'Country','Market', 'index'])
ML_df.head()

,Ship Mode,Segment,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit,Shipping Cost,...,Days to Ship,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Order Year,Order Month
0,First Class,Consumer,Central US,Technology,Phones,221.98,2,0.0,62.15,40.77,...,2,0,1,0,0,0,0,0,2014,11
1,Second Class,Corporate,Oceania,Furniture,Chairs,3709.40,9,0.1,-288.77,923.63,...,2,0,0,1,0,0,0,0,2014,2
2,First Class,Consumer,Oceania,Technology,Phones,5175.17,9,0.1,919.97,915.49,...,1,0,0,0,0,1,0,0,2014,10
3,First Class,Home Office,Western Europe,Technology,Phones,2892.51,5,0.1,-96.54,910.16,...,2,0,1,0,0,0,0,0,2014,1
4,Same Day,Consumer,Western Africa,Technology,Copiers,2832.96,8,0.0,311.52,903.04,...,1,0,0,1,0,0,0,0,2014,11


In [10]:
# encode Ship Mode, Segment, Region, Sub/Category, Order Priority
ML_df = pd.get_dummies(ML_df, columns=['Ship Mode', 'Segment', 'Region', 'Category', 'Sub-Category', 'Order Priority'])
ML_df.head()

,Sales,Quantity,Discount,Profit,Shipping Cost,Returned,Profit Margin Percentage,Returned $ Amount,Days to Ship,Monday,...,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Order Priority_Critical,Order Priority_High,Order Priority_Low,Order Priority_Medium
0,221.98,2,0.0,62.15,40.77,0.0,27.998018,0.0,2,0,...,0,0,1,0,0,0,0,1,0,0
1,3709.40,9,0.1,-288.77,923.63,0.0,-7.784817,0.0,2,0,...,0,0,0,0,0,0,1,0,0,0
2,5175.17,9,0.1,919.97,915.49,0.0,17.776614,0.0,1,0,...,0,0,1,0,0,0,0,0,0,1
3,2892.51,5,0.1,-96.54,910.16,0.0,-3.337586,0.0,2,0,...,0,0,1,0,0,0,0,0,0,1
4,2832.96,8,0.0,311.52,903.04,0.0,10.996272,0.0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
# save the ML prepped ML_df to a csv
ML_df.to_csv('../resources/superstore_ML_prepped.csv')

# Linear Regression

### Sales

In [12]:
# set target (sales) and features
y = ML_df['Sales']
X = ML_df.drop(columns='Sales')

# get columns to create coefficient df
sales_columns = X.columns
sales_columns = pd.Series(sales_columns)

In [13]:
# split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
                                                    #stratify=y)
X_train.shape

(38467, 71)

In [14]:
# create a LinearRegression instance
model = LinearRegression()

In [15]:
# fit the model to the train set
model.fit(X_train, y_train)

LinearRegression()

In [16]:
# have the model predict the y values
y_pred = model.predict(X_test)

In [17]:
# examine the coefficients and the intercept - create series for both
sales_coeffs = model.coef_
sales_coeffs = pd.Series(sales_coeffs)

sales_intercept = model.intercept_

In [18]:
# create df with variables and its coefficient
data = {"Variables" : sales_columns,
       "Coefficients" : sales_coeffs}
sales_coeff_df = pd.concat(data, axis=1)

# force python to show max rows
pd.set_option('display.max_rows', None)
sales_coeff_df

,Variables,Coefficients
0,Quantity,2.616921e+01
1,Discount,-7.901536e+01
2,Profit,6.417341e-01
3,Shipping Cost,5.059989e+00
4,Returned,-7.540537e+01
5,Profit Margin Percentage,-7.859676e-01
6,Returned $ Amount,2.753622e-01
7,Days to Ship,1.264029e-01
8,Monday,-2.381279e+11
9,Tuesday,-2.381279e+11


In [19]:
print(f' The intercept for this model is {sales_intercept}')

 The intercept for this model is 1657811840257.3862


In [20]:
# calculate the r-squared
r2_lin_sales = r2_score(y_test, y_pred)
print(f'The r2 score for the linear regression model for sales is {r2_lin_sales}')

The r2 score for the linear regression model for sales is 0.7038316921464297


### Profit

In [21]:
# set target (profit) and features
y = ML_df['Profit']
X = ML_df.drop(columns='Profit')

# get columns to create coefficient df
profit_columns = X.columns
profit_columns = pd.Series(profit_columns)

In [22]:
# split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
                                                    #stratify=y)
X_train.shape

(38467, 71)

In [23]:
# create a LinearRegression instance
model = LinearRegression()

In [24]:
# fit the model to the train set
model.fit(X_train, y_train)

LinearRegression()

In [25]:
# have the model predict the y values
y_pred = model.predict(X_test)

In [26]:
# examine the coefficients and the intercept - create series for both
profit_coeffs = model.coef_
profit_coeffs = pd.Series(profit_coeffs)

profit_intercept = model.intercept_

In [27]:
# create df with variables and its coefficient
data = {"Variables" : profit_columns,
       "Coefficients" : profit_coeffs}
profit_coeff_df = pd.concat(data, axis=1)

# force python to show max rows
pd.set_option('display.max_rows', None)
profit_coeff_df

,Variables,Coefficients
0,Sales,1.646364e-01
1,Quantity,-4.436546e+00
2,Discount,2.828323e+01
3,Shipping Cost,1.417407e-01
4,Returned,-1.712442e+01
5,Profit Margin Percentage,1.391111e+00
6,Returned $ Amount,7.742088e-02
7,Days to Ship,7.940410e-01
8,Monday,1.909220e+10
9,Tuesday,1.909220e+10


In [28]:
print(f' The intercept for this model is {profit_intercept}')

 The intercept for this model is 239698565265.81296


In [29]:
# calculate the r-squared
r2_lin_profit = r2_score(y_test, y_pred)
print(f'The r2 score for the linear regression model for profit is {r2_lin_profit}')

The r2 score for the linear regression model for profit is 0.3655354942459712


### Shipping Cost

In [30]:
# set target (shipping cost) and features
y = ML_df['Shipping Cost']
X = ML_df.drop(columns='Shipping Cost')

# get columns to create coefficient df
shipping_columns = X.columns
shipping_columns = pd.Series(shipping_columns)

In [31]:
# split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
                                                    #stratify=y)
X_train.shape

(38467, 71)

In [32]:
# create a LinearRegression instance
model = LinearRegression()

In [33]:
# fit the model to the train set
model.fit(X_train, y_train)

LinearRegression()

In [34]:
# have the model predict the y values
y_pred = model.predict(X_test)

In [35]:
# examine the coefficients and the intercept - create series for both
shipping_coeffs = model.coef_
shipping_coeffs = pd.Series(shipping_coeffs)

shipping_intercept = model.intercept_

In [36]:
# create df with variables and its coefficient
data = {"Variables" : shipping_columns,
       "Coefficients" : shipping_coeffs}
shipping_coeff_df = pd.concat(data, axis=1)

# force python to show max rows
pd.set_option('display.max_rows', None)
shipping_coeff_df

,Variables,Coefficients
0,Sales,0.083824
1,Quantity,1.262951
2,Discount,-3.323946
3,Profit,0.009152
4,Returned,1.955880
5,Profit Margin Percentage,-0.003952
6,Returned $ Amount,-0.002139
7,Days to Ship,-0.327681
8,Monday,-0.184250
9,Tuesday,-0.173443


In [37]:
print(f' The intercept for this model is {shipping_intercept}')

 The intercept for this model is -40.85701372876658


In [38]:
# calculate the r-squared
r2_lin_shipping = r2_score(y_test, y_pred)
print(f'The r2 score for the linear regression model for shipping costs is {r2_lin_shipping}')

The r2 score for the linear regression model for shipping costs is 0.6029476282063628


# Logistic Regression

### Returned?

In [39]:
# set target (Returned) and features
y = ML_df['Returned']
X = ML_df.drop(columns=['Returned', 'Returned $ Amount'])

# get columns to create coefficient df
returned_columns = X.columns
returned_columns = pd.Series(returned_columns)

In [40]:
# split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
                                                    #stratify=y)
returned_split = Counter(y_train)
returned_split

Counter({0.0: 36805, 1.0: 1662})

In [41]:
# create the log regression model 
model = LogisticRegression()

In [42]:
# Train the data
model.fit(X_train, y_train)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [43]:
# Predict outcomes for test data set
y_pred = model.predict(X_test)
pred_actual = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})

In [44]:
# examine the coefficients and the intercept - create series for both
returned_coeffs = model.coef_
returned_coeffs = pd.Series(returned_coeffs.flatten())

returned_intercept = model.intercept_

In [45]:
# create df with variables and its coefficient
data = {"Variables" : returned_columns,
       "Coefficients" : returned_coeffs}
returned_coeff_df = pd.concat(data, axis=1)

# force python to show max rows
pd.set_option('display.max_rows', None)
returned_coeff_df

,Variables,Coefficients
0,Sales,-0.000309
1,Quantity,0.005771
2,Discount,0.006325
3,Profit,0.000058
4,Shipping Cost,0.001974
5,Profit Margin Percentage,0.000492
6,Days to Ship,0.043067
7,Monday,0.003830
8,Tuesday,0.002987
9,Wednesday,0.013085


In [46]:
print(f' The intercept for this model is {returned_intercept}')

 The intercept for this model is [7.58464319e-06]


In [47]:
confusion_matrix(y_test, y_pred)

array([[12265,     0],
       [  558,     0]], dtype=int64)

In [48]:
# using balanced accuracy score since this data is so un-balanced
acc_score = balanced_accuracy_score(y_test, y_pred)
print(f'The balanced accuracy score for the un-altered logistic regression is {acc_score}')

The balanced accuracy score for the un-altered logistic regression is 0.5


### SMOTE

In [49]:
# SMOTE the train data sets
X_SMOTE, y_SMOTE = SMOTE(sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_SMOTE)

Counter({0.0: 36805, 1.0: 36805})

In [50]:
# initiate new model & fit to the randomly oversampled train data
model= LogisticRegression()
model.fit(X_SMOTE, y_SMOTE)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [51]:
# examine the coefficients and the intercept - create series for both
returned_smote_coeffs = model.coef_
returned_smote_coeffs = pd.Series(returned_smote_coeffs.flatten())

returned_smote_intercept = model.intercept_

In [52]:
# create df with variables and its coefficient
data = {"Variables" : returned_columns,
       "Coefficients" : returned_smote_coeffs}
returned_smote_coeff_df = pd.concat(data, axis=1)

# force python to show max rows
pd.set_option('display.max_rows', None)
returned_smote_coeff_df

,Variables,Coefficients
0,Sales,0.001218
1,Quantity,-0.437217
2,Discount,0.013837
3,Profit,-0.000857
4,Shipping Cost,-0.005129
5,Profit Margin Percentage,0.002682
6,Days to Ship,-0.368588
7,Monday,-0.035288
8,Tuesday,-0.274647
9,Wednesday,-0.265304


In [53]:
print(f' The intercept for this model is {returned_smote_intercept}')

 The intercept for this model is [0.00034388]


In [54]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[8742, 3523],
       [ 400,  158]], dtype=int64)

In [55]:
# using accuracy score since data is now balanced
smote_acc_score = accuracy_score(y_test, y_pred)
print(f'The accuracy score for the SMOTE-sampled logistic regression is {smote_acc_score}')

The accuracy score for the SMOTE-sampled logistic regression is 0.6940653513218435


### SMOTEENN

In [56]:
# create instance of SMOTEENN and resample train data 
smote_enn = SMOTEENN()
X_SMOTEENN, y_SMOTEENN = smote_enn.fit_resample(X_train, y_train)
Counter(y_SMOTEENN)

Counter({0.0: 22872, 1.0: 34931})

In [57]:
# initiate new model & fit to the resampled train data
model= LogisticRegression()
model.fit(X_SMOTEENN, y_SMOTEENN)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [58]:
# examine the coefficients and the intercept - create series for both
returned_smoteenn_coeffs = model.coef_
returned_smoteenn_coeffs = pd.Series(returned_smoteenn_coeffs.flatten())

returned_smoteenn_intercept = model.intercept_

In [59]:
# create df with variables and its coefficient
data = {"Variables" : returned_columns,
       "Coefficients" : returned_smoteenn_coeffs}
returned_smoteenn_coeff_df = pd.concat(data, axis=1)

# force python to show max rows
pd.set_option('display.max_rows', None)
returned_smoteenn_coeff_df

,Variables,Coefficients
0,Sales,0.000768
1,Quantity,-0.439707
2,Discount,0.015826
3,Profit,-0.000609
4,Shipping Cost,-0.002569
5,Profit Margin Percentage,0.003809
6,Days to Ship,-0.281548
7,Monday,-0.031887
8,Tuesday,-0.255857
9,Wednesday,-0.251745


In [60]:
print(f' The intercept for this model is {returned_smoteenn_intercept}')

 The intercept for this model is [0.00034486]


In [61]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[6409, 5856],
       [ 280,  278]], dtype=int64)

In [62]:
# using accuracy score since data is now balanced
smoteenn_acc_score = accuracy_score(y_test, y_pred)
print(f'The accuracy score for the randomly SMOTEENN-sampled logistic regression is {smoteenn_acc_score}')

The accuracy score for the randomly SMOTEENN-sampled logistic regression is 0.5214848319426031


# Results

In [63]:
print(f'The r2 score for the linear regression model for sales is {r2_lin_sales}')
print(f'The r2 score for the linear regression model for profit is {r2_lin_profit}')
print(f'The r2 score for the linear regression model for shipping costs is {r2_lin_shipping}\n')
print(f'The balanced accuracy score for the un-altered logistic regression is {acc_score}')
print(f'The split between not returned (0) and returned (1) for the un-altered regression is {returned_split}\n')
print(f'The accuracy score for the SMOTE-sampled logistic regression is {smote_acc_score}')
print(f'The accuracy score for the SMOTEENN-sampled logistic regression is {smoteenn_acc_score}')

The r2 score for the linear regression model for sales is 0.7038316921464297
The r2 score for the linear regression model for profit is 0.3655354942459712
The r2 score for the linear regression model for shipping costs is 0.6029476282063628

The balanced accuracy score for the un-altered logistic regression is 0.5
The split between not returned (0) and returned (1) for the un-altered regression is Counter({0.0: 36805, 1.0: 1662})

The accuracy score for the SMOTE-sampled logistic regression is 0.6940653513218435
The accuracy score for the SMOTEENN-sampled logistic regression is 0.5214848319426031
